# Bring Your Own Container (BYOC)

In this notebook, we will cover how to bring our own container with either a framework or algorithm to train a model on SageMaker. 

We will use fastai in this case and build our container with custom training code integrated into the container. The other option is to use script mode which is easily done by changing the entrypoint.

Outline:
1. Build your own custom docker container with fastai framework
2. Register your custom docker container to Elastic Container Registry (ECR)
3. Use your custom docker container and provided training dataset to train your model


---
### 1. Build you customer Docker Container Image
Let's start with building a container image locally and then push that to ECR (Elastic Container Registry)

In [1]:
%cd docker

/home/ec2-user/SageMaker/sagemaker-scriptmode-byoc-workshop/fastai-rf/docker


In [2]:
# let review the Dockerfile used to build your custom container iage
!pygmentize Dockerfile

# Part of the implementation of this container is based on the Amazon SageMaker Apache MXNet container.
# https://github.com/aws/sagemaker-mxnet-container

FROM fastdotai/fastai:latest

LABEL maintainer="Raj Kadiyala"


WORKDIR /

# Installing numpy, pandas, scikit-learn, scipy
RUN pip3 install --no-cache --upgrade requests 

# Setting some environment variables.
ENV PYTHONDONTWRITEBYTECODE=1 \
    PYTHONUNBUFFERED=1 \
    LD_LIBRARY_PATH="${LD_LIBRARY_PATH}:/usr/local/lib" \
    PYTHONIOENCODING=UTF-8 \
    LANG=C.UTF-8 \
    LC_ALL=C.UTF-8

RUN pip3 install --no-cache --upgrade \
    sagemaker-training

# Copies code under /opt/ml/code where sagemaker-containers expects to find the script to run
COPY code/* /opt/ml/code/
# RUN pip uninstall urllib3
# RUN pip uninstall requests
# RUN pip uninstall chardet
# RUN yum remove python-requests
# RUN yum remove python-urllib3
# RUN pip install --upgrade --force-reinstall 'requests==2.6.0' urllib3
# RUN yum install certbot
# Defines train.py 

In [3]:
# let review training script
!pygmentize code/train.py

from __future__ import absolute_import
from fastai.vision.all import *
from fastai.metrics import error_rate, accuracy
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path
# Set path to root directory
import argparse
import os
import sys
import time
from utils import print_files_in_path, save_model_artifacts


def train(lr, train_channel):
    
    path = Path(os.environ["SM_CHANNEL_TRAIN"])
    dblock = DataBlock(blocks = (ImageBlock, CategoryBlock),
               get_items = get_image_files,
               splitter = GrandparentSplitter(),
               get_y = parent_label)
    
    
    dls=dblock.dataloaders(path)
    
    print(dls.vocab)
    # Dummy net.
    net=cnn_learner(dls,resnet50,metrics=error_rate)
    

    # Run training loop.
    net.fit_one_cycle(5, 1e-03)

    # At the end of the training loop, we have to save model artifacts.
    model_dir = os.environ["SM_MODEL_DIR"]
    net.export(model_dir + "/export.pkl")


if __name__ == "__main__":

  

In [4]:
# let review utils script
!pygmentize code/utils.py

import os
from os import path


def save_model_artifacts(model_artifacts_path, net):
    if path.exists(model_artifacts_path):
        model_file = open(model_artifacts_path + "model.dummy", "w")
        model_file.write("Dummy model.")
        model_file.close()


def print_files_in_path(path):
    files = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            files.append(os.path.join(r, file))

    for f in files:
        print(f)


In [5]:
# build the docker image
!docker build -t fastai .

Sending build context to Docker daemon   12.8kB
Step 1/8 : FROM fastdotai/fastai:latest
latest: Pulling from fastdotai/fastai

55322776: Pulling fs layer 
02cb4e21: Pulling fs layer 
b1d03d39: Pulling fs layer 
e659dcd9: Pulling fs layer 
3087fc87: Pulling fs layer 
f9a9dc88: Pulling fs layer 
e409dd55: Pulling fs layer 
a6f1de28: Pulling fs layer 
be2dfbaf: Pulling fs layer 
b700ef54: Pulling fs layer 
24259e5f: Pulling fs layer 
0e44e022: Pulling fs layer 
07d9cb8d: Pulling fs layer 
b2663a0b: Pulling fs layer 
Digest: sha256:d8464f4af21d1dadefda33e07f4ffb854c821fec530efe35c7e30b66b338ee02Downloading  655.7MB/2.962GBExtracting  885.7MB/2.962GBExtracting  1.486GB/2.962GB
Status: Downloaded newer image for fastdotai/fastai:latest
 ---> 2fe7a9598c6e
Step 2/8 : LABEL maintainer="Raj Kadiyala"
 ---> Running in e76413f95761
Removing intermediate container e76413f95761
 ---> 11dd51357cf2
Step 3/8 : WORKDIR /
 ---> Running in 5d9c8943caf4
Removing intermediate container 5d9c8943caf4
 ---> 8d

Add docker images architecture to explain what is going on
show the Dockerfile

In [6]:
# show the docker image
!docker images

REPOSITORY         TAG       IMAGE ID       CREATED                  SIZE
fastai             latest    c9f15abdfdab   Less than a second ago   8.23GB
fastdotai/fastai   latest    2fe7a9598c6e   15 hours ago             8.2GB


---
## 2. Register your custom docker image in ECR

Set the ecr details and tags - Lets set a few params here like ecr name space , tag name etc.

In [7]:
from sagemaker import get_execution_role
import boto3
ecr_namespace = "sagemaker-training-containers/"
prefix = "byoc-fastai"

ecr_repository_name = ecr_namespace + prefix
role = get_execution_role()
account_id = role.split(":")[4]
region = boto3.Session().region_name
tag_name=account_id+'.dkr.ecr.'+region+'.amazonaws.com/'+ecr_repository_name+':latest'

In [8]:
# review the tag name
tag_name

'406078665760.dkr.ecr.us-east-1.amazonaws.com/sagemaker-training-containers/byoc-fastai:latest'

In [9]:
# tag the image 
!docker tag fastai $tag_name

### ECR Repository and push steps

All of these can be scripted out but they are laid out this way for transparency and step evolution understanding

In [10]:
# first login to ECR
!$(aws ecr get-login --no-include-email)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [11]:
# create a repository 
!aws ecr create-repository --repository-name $ecr_repository_name


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'sagemaker-training-containers/byoc-fastai' already exists in the registry with id '406078665760'


In [12]:
# now push the image to ECR
!docker push $tag_name

The push refers to repository [406078665760.dkr.ecr.us-east-1.amazonaws.com/sagemaker-training-containers/byoc-fastai]

a78c27f7: Preparing 
734ad6b1: Preparing 
948180d1: Preparing 
8412ba88: Preparing 
6e7f7c50: Preparing 
02edd6e9: Preparing 
d354551c: Preparing 
6b782713: Preparing 
bf18a086: Preparing 
64a0f101: Preparing 
cd86775d: Preparing 
0b5090fa: Preparing 
e3d203f2: Preparing 
eb8ce83e: Preparing 
1b69538d: Preparing 
fcbe6a14: Preparing 
0881ce19: Preparing 
0b5090fa: Pushed   1.475GB/1.452GB[18APushing    652MB/1.452GBlatest: digest: sha256:0450e7b8f1b5e374560ab74bce0729062adc68c3c919312087e7b74e2e9c420e size: 4295


In [13]:
# quick check 
container_image_uri = "{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest".format( account_id, region, ecr_repository_name)
print(container_image_uri)

406078665760.dkr.ecr.us-east-1.amazonaws.com/sagemaker-training-containers/byoc-fastai:latest


In [14]:
my_bucket = "2021-11-vilas123456"
#!aws s3 mb 's3://'$my_bucket --region us-east-1

In [15]:
# review the prepared training data
!aws s3 ls 's3://'$my_bucket/train --region us-east-1

                           PRE train/


---
### 3. Now Call your custom container to train the model

In [ ]:
# let train a model
import sagemaker
import json

role = sagemaker.get_execution_role()

# replace the s3_bucket name with your bucket name
s3_bucket = 's3://'+my_bucket+'/train'

# JSON encode hyperparameters
def json_encode_hyperparameters(hyperparameters):
    return {str(k): json.dumps(v) for (k, v) in hyperparameters.items()}


hyperparameters = json_encode_hyperparameters({"lr":1e-03})

est = sagemaker.estimator.Estimator(
    container_image_uri,
    role,
    instance_count=1,
    #train_instance_type="local",  # we use local mode
    instance_type='ml.m5.4xlarge',
    base_job_name=prefix,
    hyperparameters=hyperparameters,
)

train_config = sagemaker.session.TrainingInput(s3_bucket)

est.fit({"train": train_config})

2021-11-24 21:10:34 Starting - Starting the training job...
2021-11-24 21:10:37 Starting - Launching requested ML instancesProfilerReport-1637788234: InProgress
......
2021-11-24 21:11:53 Starting - Preparing the instances for training.........
2021-11-24 21:13:34 Downloading - Downloading input data...
2021-11-24 21:13:54 Training - Downloading the training image.....................
2021-11-24 21:17:36 Training - Training image download completed. Training in progress..2021-11-24 21:17:30,630 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-11-24 21:17:36,895 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-11-24 21:17:36,906 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-11-24 21:17:36,931 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/dat

## Your model is now ready to be deployed